In [2]:
!git clone https://github.com/WaShindeiru/IceCubeNeutrino

Cloning into 'IceCubeNeutrino'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 83 (delta 42), reused 27 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (83/83), 541.81 KiB | 5.58 MiB/s, done.
Resolving deltas: 100% (42/42), done.


In [3]:
!cp IceCubeNeutrino/* .

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Learning rate mniejszy, funkcja straty inna, residuum, porównanie przewidywanej z teoretyczną, porównanie walidacyjną z treningową.

In [ ]:
!unzip ./drive/MyDrive/icecube/ice_cube_small.zip

In [1]:
import pandas as pd
import pyarrow.parquet as pq
from tqdm import tqdm
import torch
import torch.nn as nn
from Resnet import resnet34

ModuleNotFoundError: No module named 'Resnet'

In [ ]:
sensor = pd.read_csv('./ice_cube_small/sensor_geometry.csv')
train_meta = pq.ParquetFile('./ice_cube_small/train_meta.parquet')
batch_1 = pq.read_table('./ice_cube_small/train_2/batch_1.parquet').to_pandas()
it = train_meta.iter_batches()
train_meta = next(it).to_pandas()

In [ ]:
train_meta.head(20)

In [ ]:
sensor.head(20)

In [ ]:
batch_1

In [ ]:
import torch.nn as nn
model = resnet34(pretrained = False)
model.fc = nn.Sequential(nn.ReLU(),nn.Linear(in_features=512, out_features=2))

In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np

In [ ]:
from torch import optim
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = 'cpu'


pqfile = './ice_cube_small/train_meta.parquet'
path_batch = './ice_cube_small/train_2/'
geometry = "./ice_cube_small/sensor_geometry.csv"

batch_num=2
# lepszy learning rate
# mniejszy rozmiar

lr = 1e-06
num_epochs = 1
batch_size = 4

model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
loss_function = nn.MSELoss().to(device)

In [ ]:
from torch.utils.data import DataLoader
from Icecube_Dataloader import IceCube_Dataloader, collate_fn

ice_dataset = IceCube_Dataloader(pqfile, path_batch, geometry, batch_num)
train_dataset = ice_dataset

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn, num_workers=11)

In [ ]:
print(device)

In [ ]:
params = {
    'num_epochs':num_epochs,
    'optimizer':optimizer,
    'loss_function':loss_function,
    'train_dataloader':train_dataloader,
    'device':device,
    'num_epoch' : num_epochs
}


In [ ]:
import gc
train_losses = []
outputs_ = []
accum_iter = 1

def train(model, params):
    model.train()
    loss_function=params["loss_function"]
    train_dataloader=params["train_dataloader"]

    aa = 0

    device=params["device"]
    for epoch in range(0, num_epochs):
        for dat in tqdm(train_dataloader):
            inputs, labels = dat['input_tensor'].to(device),dat['label'].to(device)

            assert not torch.isnan(inputs).any(), "Inputs contain NaNs"
            assert not torch.isnan(labels).any(), "Labels contain NaNs"

            outputs = model(inputs).to(device)
            outputs_.append(outputs.cpu().detach().numpy())

            train_loss = loss_function(outputs.float(),labels.float())
            train_losses.append(train_loss.item())


            if aa%100==0:
                print(f"Train loss: {train_loss.item()}")

            train_loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            optimizer.zero_grad()
            torch.cuda.empty_cache()

            aa = aa+1

    model.eval()
    gc.collect()

    torch.cuda.empty_cache()
    return 0


train(model, params)

In [ ]:
import matplotlib.pyplot as plt


plt.figure()
plt.plot(train_losses)

In [ ]:
from datetime import datetime

now = datetime.now()

timestamp = now.strftime("%Y-%m-%d_%H:%M:%S")

In [ ]:
name = "./weights/model_34_"+timestamp+".pth"
torch.save(model.state_dict(), name)

In [ ]:
import pickle

name = "./results/losses_"+timestamp+".pickle"
with open(name, 'wb') as f:
    pickle.dump(train_losses, f)